In [ ]:
import os, sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp

# modelica interface
import buildingspy.simulate.Simulator as si
import buildingspy.io.outputfile as buipy_io

# thermal
import  CoolProp.CoolProp as CP
from CoolProp.HumidAirProp import HAPropsSI
import psychrolib as psylib
psylib.SetUnitSystem(psylib.SI)

In [ ]:
sys.path.insert(0, os.path.join('.','..','calibration'))
from thermodynamics import *
from postprocessing import *
from plotting import *
from calibration_MgCl2_class import *

# import code of mathieu
import media_model_mgcl2 as model_mg
import media_model_water as model_w

In [ ]:
%load_ext autoreload
# only reload the function imported by "aimport"
%autoreload 1
%aimport thermodynamics, postprocessing, plotting, calibration_MgCl2_class

# Data preprocessing

In [ ]:
path_expdata = os.path.join(os.path.expanduser('~'), 'GIT', 'TheGreeFa_EnBA', 'Data_input', 'measurement')

## Chen2016

In [ ]:
df_chen_data = pd.read_excel(os.path.join(path_expdata, "actual_data_converted/Chen2016.xlsx"), index_col=0)

In [ ]:
df_chen_data['x_d_in_equ'] = np.vectorize(calc_x_des_equ)(df_chen_data.T_d_in, df_chen_data.xi_d_in)
df_chen_data["RH_a_in"] = np.vectorize(get_RH)(df_chen_data.T_a_in, df_chen_data.x_a_in) *100
df_chen_data["RH_a_o_exp"] = np.vectorize(get_RH)(df_chen_data.T_a_o_exp, df_chen_data.x_a_o_exp) *100
df_chen_data["RH_d_in_equ"] = np.vectorize(get_RH)(df_chen_data.T_d_in, df_chen_data.x_d_in_equ) *100

In [ ]:
plot_with_des_point_df(df_data=df_chen_data, chart_template='minimal', use_line_highlight = False, ncol = 3, ) # save_fig_name="Chen_minimal.png"

## Dempav

In [ ]:
df_exp_dempav = pd.read_excel(os.path.join(path_expdata, "actual_data_converted/dempav_T_des_corrected.xlsx"), index_col=0)
df_exp_dempav.head()

In [ ]:
df_exp_dempav['x_d_in_equ'] = np.vectorize(calc_x_des_equ)(df_exp_dempav.T_d_in, df_exp_dempav.xi_d_in)
df_exp_dempav["RH_a_in"] = df_exp_dempav["RH_a_in"] * 100
df_exp_dempav["RH_a_o_exp"] = df_exp_dempav["RH_a_o_exp"] * 100
df_exp_dempav["RH_d_in_equ"] = np.vectorize(get_RH)(df_exp_dempav.T_d_in, df_exp_dempav.x_d_in_equ) *100

In [ ]:
plot_air_state_df(df_data=df_exp_dempav, chart_template="minimal", alpha=0.8, )  # save_fig_name='dempav_all.png'

In [ ]:
df_exp_dempav.query('T_a_in - T_a_o_exp > 2').shape

In [ ]:
plot_with_des_point_df(df_data=df_exp_dempav.query('T_a_in - T_a_o_exp > 2'), chart_template="minimal", alpha=0.8, save_fig_name='dempav_cooling.png') # save_fig_name='dempav_cooling.png' 

# Calculate Nu/Sh through calibration
- the main code for this section can be found in calibration_MgCl2_class.py

## Chen2016

In [ ]:
def run_calib_LiCl(prefix_output, file_name):
    bib_path = os.path.join(os.path.expanduser("~"),"GIT", 'TheGreeFa_EnBA')
    model_path = 'BrineGrid.Fluid.Absorbers.Examples.Validation.'
    
    # run chen2016
    model_name = 'Calibration_Chen_2016'
    print("*****start {}******".format(model_name))

    df = pd.read_excel(os.path.join(bib_path, "Data_input", "measurement", "actual_data_converted/{}".format(file_name)),\
                       index_col=0)
    print(df)

    # generate object for calibration class
    
    LiCl_obj = Calibration_H_M_transfer(t_stop=3600, exchange_area=450*1.5*0.75*0.75, d_e=0.01, type_desiccant='LiCl',\
        cross_area=1.5*0.75, void_fraction = 0.9,bib_path=bib_path, model_path = model_path, nNodes=8, mNodes=8, \
            model_name=model_name, prefix_outputfolder=prefix_output, flow_config = 'cross')

    output_path = LiCl_obj.get_output_folder()

    # # run multiple calibration
    LiCl_obj.mp_run(df=df, num_core=mp.cpu_count()-2)
    # # run single calibration
    # LiCl_obj.run_calibration(input_dataset=df.iloc[9,:])

    df_output = LiCl_obj.read_results(path=output_path)
    df_output.to_csv(path_or_buf=os.path.join(output_path, 'final_output.csv'))

    print('{} finished'.format(model_name))
    return

In [ ]:
run_calib_LiCl("New_Git_", "Chen2016.xlsx")

## Dempav

In [ ]:
def run_calib_MgCl2_dempav(prefix_output, file_name):
    # # run MgCl2
    model_name = 'Calibration_MgCl2_dempav'
    print("*****start {}******".format(model_name))

    bib_path = os.path.join(os.path.expanduser("~"),"GIT",'TheGreeFa_EnBA')
    model_path = 'BrineGrid.Fluid.Absorbers.Examples.Validation.'

    df = pd.read_excel(os.path.join(bib_path, "Data_input", "measurement", "actual_data_converted/{}").format(file_name),index_col = 0) .query('T_a_in - T_a_o_exp > 2')
    print(df)
    
    # # generate object for calibration class
    MgCl2_obj = Calibration_H_M_transfer(t_stop=3600, exchange_area=6.5, d_e=0.04, cross_area = 0.065, \
        void_fraction = 0.95, delta_T = 0.5, bib_path=bib_path, model_path = model_path, \
            nNodes=8, mNodes=8, model_name=model_name, prefix_outputfolder=prefix_output)
    
    output_path = MgCl2_obj.get_output_folder()
    print("output path is: {}.".format(output_path))
    
    try:
        MgCl2_obj.run_calibration(input_dataset=df.iloc[8,:]) # single simulation
        # MgCl2_obj.mp_run(df=df, num_core=8) #mp.cpu_count()- 4
    except:
        print('Calibration for {} + {} didn\'t completely finished.'.format(model_name, file_name))

    df_output = MgCl2_obj.read_results(path=output_path)
    df_output.to_csv(path_or_buf=os.path.join(output_path, 'final_output.csv'))

    print('{} (dempav) finished'.format(model_name))
    return

In [ ]:
run_calib_MgCl2_dempav("2021_oct_dempav_3_", "dempav_T_des_corrected.xlsx")

# Get Nu/Sh formula through regression
- the main code for this section can be found in calibration_MgCl2_class.py

- Note: the parameter a-f calculated through regression should be kept as small as possbile (-2 to +2 recommended) to avoid the deviation caused by expoential calculation

## Chen2016

In [ ]:
# check results of last step:
bath_path = os.path.join(os.path.expanduser("~"),"GIT","TheGreeFa_EnBA", "Results", "modelica")
folder_name = "2021_oct_Calibration_Chen_2016"
file_name = "final_output.csv"

df = pd.read_csv(os.path.join(bath_path, folder_name, file_name), index_col=0)
df

In [ ]:
# declare object
obj_postproc = EnBA_M_post_processing(folder_name = "2021_oct_Calibration_Chen_2016", file_name = "final_output.csv")

# run regression
obj_postproc.run_regression_generate_results(method="scipy", Nu_num_training=12, \
                                               Nu_if_sorted=True, Sh_num_training=12, \
                                               Sh_if_sorted=False, physical_Nu_Sh=False, if_plot=True, max_iter=10000)

## Dempav

In [ ]:
# check results of last step:
bath_path = os.path.join(os.path.expanduser("~"),"GIT","TheGreeFa_EnBA", "Results", "modelica")
folder_name = "2021_oct_dempav_Calibration_MgCl2_dempav"
file_name = "final_output.csv"

df = pd.read_csv(os.path.join(bath_path, folder_name, file_name), index_col=0)
df

In [ ]:
# declare object
obj_postproc = EnBA_M_post_processing(folder_name = "2021_oct_dempav_Calibration_MgCl2_dempav", file_name = "final_output.csv")

# run regression
obj_postproc.run_regression_generate_results(method="scipy", Nu_num_training=12, \
                                               Nu_if_sorted=False, Sh_num_training=12, \
                                               Sh_if_sorted=False, physical_Nu_Sh=False, if_plot=True, max_iter=10000)

# run simulation with new Nu-Sh Formular (calculated by regression)
- Firstly, the formular needs to be passed into modelica manually

In [ ]:
def run_Chen_fomular_NuSh(prefix_output, file_name):
    # run simulation using implemented formular for Nu&Sh in Modelica #
    model_name = 'Validation_new_formular_Chen_2016'
    print("*****start {}******".format(model_name))

    bib_path = os.path.join(os.path.expanduser("~"),"GIT",'TheGreeFa_EnBA')
    model_path = 'BrineGrid.Fluid.Absorbers.Examples.Validation.'

    df = pd.read_excel(os.path.join(bib_path, "Data_input", "measurement", "actual_data_converted/{}".format(file_name)),\
                       index_col = 0)
    print(df)

    # # generate object for calibration class
    LiCl2_obj = Calibration_H_M_transfer(t_stop=1800, exchange_area=450*1.5*0.75*0.75, type_desiccant='LiCl',\
        d_e=0.01, cross_area = 1.5*0.75, void_fraction = 0.9, bib_path=bib_path, model_path = model_path, flow_config = 'cross', \
        nNodes=10, mNodes=10, model_name=model_name, prefix_outputfolder=prefix_output, if_Nu_Sh_const=False)

    output_path = LiCl2_obj.get_output_folder()
    
    print("output path is: {}.".format(output_path))
    
    # LiCl2_obj.run_single_simulation(input_dataset=df.iloc[0,:])
    LiCl2_obj.mp_run(df=df, mode="simulation", num_core=mp.cpu_count()-2)

    df_output = LiCl2_obj.read_results(path=output_path, mode="simulation")
    df_output.to_csv(path_or_buf=os.path.join(output_path, 'final_output.csv'))

    print('{} finished'.format(model_name))
    return

In [ ]:
# run_Chen_fomular_NuSh(prefix_output='Chen_regress_fomular_Nu_Sh_2021Nov_2_', file_name='Chen2016.xlsx')

# Validate the results

## Chen2016

### read simulation results

In [ ]:
bath_path = os.path.join(os.path.expanduser("~"),"GIT","TheGreeFa_EnBA", "Results", "modelica")
folder_name = "Chen_regress_fomular_Nu_Sh_2021Nov_2_Calibration_Chen_2016"
file_name = "final_output.csv"

df_sim_results = pd.read_csv(os.path.join(bath_path, folder_name, file_name), index_col=0)
df_sim_results

### Model results verification by comparing with expdata and plotting

In [ ]:
df_sim_results["delta_T_a_exp"] =  df_sim_results.T_a_in - df_sim_results.T_a_o_exp
df_sim_results["delta_x_a_exp"] =  df_sim_results.x_a_in - df_sim_results.x_a_o_exp

In [ ]:
df_sim_results["delta_T_a_sim"] =  df_sim_results.T_a_in - df_sim_results.T_a_o_sim_reg
df_sim_results["delta_x_a_sim"] =  df_sim_results.x_a_in - df_sim_results.x_a_o_sim_reg

In [ ]:
df_results_compare = df_sim_results[["delta_T_a_exp", "delta_x_a_exp", "delta_T_a_sim", "delta_x_a_sim"]]

In [ ]:
plot_compare_2_cols(df_results_compare.delta_T_a_exp, df_results_compare.delta_T_a_sim, deviation = 0.1, x_label = "Änderung der Lufttemp. (Messung)", y_label = "Änderung der Lufttemp. (Simulation)", save_fig="Chen_T")

In [ ]:
plot_compare_2_cols(df_results_compare.delta_x_a_exp, df_results_compare.delta_x_a_sim, deviation=0.1, x_label = "Änderung der Luftfeuchte (Messung)", y_label = "Änderung der Luftfeuchte (Simulation)", save_fig="Chen_x")

## Dempav

### read sim results

In [ ]:
bath_path = os.path.join(os.path.expanduser("~"),"GIT","TheGreeFa_EnBA", "Results", "modelica")
folder_name = "Dempav_regress_fomular_Nu_Sh_2021Nov_Validation_MgCl2_dempav"
file_name = "final_output.csv"

df_sim_results = pd.read_csv(os.path.join(bath_path, folder_name, file_name), index_col=0)
df_sim_results

In [ ]:
df_sim_results["delta_T_a_exp"] =  df_sim_results.T_a_in - df_sim_results.T_a_o_exp
df_sim_results["delta_x_a_exp"] =  df_sim_results.x_a_in - df_sim_results.x_a_o_exp
df_sim_results["delta_T_a_sim"] =  df_sim_results.T_a_in - df_sim_results.T_a_o_sim_reg
df_sim_results["delta_x_a_sim"] =  df_sim_results.x_a_in - df_sim_results.x_a_o_sim_reg

In [ ]:
df_results_compare = df_sim_results[["delta_T_a_exp", "delta_x_a_exp", "delta_T_a_sim", "delta_x_a_sim"]]

In [ ]:
plot_compare_2_cols(df_results_compare.delta_T_a_exp, df_results_compare.delta_T_a_sim, deviation = 0.2, x_label = "Änderung der Lufttemp. (Messung)", y_label = "Änderung der Lufttemp. (Simulation)", save_fig="dempav_T")

In [ ]:
plot_compare_2_cols(df_results_compare.delta_x_a_exp, df_results_compare.delta_x_a_sim, deviation=0.2, x_label = "Änderung der Luftfeuchte (Messung)", y_label = "Änderung der Luftfeuchte (Simulation)", save_fig="dempav_x")